In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
import sklearn 

# iPython command to format matplotlib plots
%matplotlib inline
from pandas.io.json import json_normalize
import json

# import ploty for visualization
import plotly
import plotly.offline as py # make offline 
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.graph_objs import *
import plotly.tools as tls
import plotly.figure_factory as fig_fact
plotly.tools.set_config_file(world_readable=True, sharing='public')
# remove war
import warnings
warnings.filterwarnings('ignore')


In [2]:
'''
load_data(filepath) 
Given a filepath to a JSON file, loads in the file and formats the JSON
'''
def load_data(filepath):
    data = []
    
    # Open file and read in line by line
    with open(filepath) as file:
        for line in file:
            # Strip out trailing whitespace at the end of the line
            data.append(json.loads(line.rstrip()))

    return data

data = load_data('checkin.json')
checkin_df = pd.DataFrame.from_dict(data)
checkin_df.head(5)

data2 = load_data('yelp.business.json')
business_df = pd.DataFrame.from_dict(data2)
business_df.head()




,_id,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,{'$oid': '5a8c8b75d42e9dd69892c4ef'},104 43rd St,"{'BusinessAcceptsCreditCards': True, 'GoodForM...",EJFdWX908N8Yc2XG0Lky8A,"[Cafes, Restaurants]",Pittsburgh,"{'Tuesday': '17:00-21:30', 'Friday': '17:00-21...",0,40.472735,-79.963265,River Moon Cafe,Lawrenceville,15201,5,4.0,PA
1,{'$oid': '5a8c8b75d42e9dd69892c4fa'},4032 Butler St,"{'GoodForMeal': {'dessert': False, 'latenight'...",o1fTwfqN0sDFNpV1CkOPPg,"[Restaurants, Sandwiches, Coffee & Tea, Food]",Pittsburgh,"{'Monday': '7:00-21:00', 'Tuesday': '7:00-21:0...",1,40.469295,-79.961273,Crazy Mocha Coffee,Lawrenceville,15201,16,3.5,PA
2,{'$oid': '5a8c8b75d42e9dd69892c4fc'},5217 Clairton Blvd,"{'GoodForMeal': {'dessert': False, 'latenight'...",-nHkhiuerqmfBG3v2v9O-g,"[Breakfast & Brunch, Bagels, Restaurants, Bake...",Pittsburgh,"{'Monday': '5:30-17:00', 'Tuesday': '5:30-17:0...",1,40.344194,-79.970963,Bruegger's Bagels,,15236,5,2.5,PA
3,{'$oid': '5a8c8b75d42e9dd69892c512'},135 9th St,"{'BusinessParking': {'garage': True, 'street':...",HcEJ2YiSLX_mT3RE0hvKWQ,"[Nightlife, Adult Entertainment, Strip Clubs]",Pittsburgh,"{'Monday': '12:00-2:00', 'Tuesday': '12:00-2:0...",1,40.444236,-79.998325,Blush,Downtown,15222,21,3.0,PA
4,{'$oid': '5a8c8b75d42e9dd69892c51d'},248 Tara Dr,"{'BusinessAcceptsCreditCards': False, 'ByAppoi...",shNBn6mTKxZ124yPC_H02Q,"[Kitchen & Bath, Countertop Installation, Home...",Pittsburgh,"{'Monday': '0:00-0:00', 'Tuesday': '0:00-0:00'...",1,40.327956,-79.960479,T.W.Tile,,15236,3,3.5,PA


In [3]:
checkin_df.head(5)

,business_id,time
0,7KPBkxAOEtb3QeIL9PEErg,"{'Thursday': {'21:00': 4, '1:00': 1, '4:00': 1..."
1,kREVIrSBbtqBhIYkTccQUg,"{'Monday': {'13:00': 1}, 'Thursday': {'20:00':..."
2,tJRDll5yqpZwehenzE2cSg,"{'Monday': {'12:00': 1, '1:00': 1}, 'Friday': ..."
3,r1p7RAMzCV_6NPF0dNoR3g,"{'Thursday': {'23:00': 1}, 'Saturday': {'21:00..."
4,mDdqgfrvROGAumcQdZ3HIg,"{'Monday': {'12:00': 1, '21:00': 1}, 'Wednesda..."


In [17]:
# Create dummy variables for categories
categories_df = business_df['categories'].str.join(sep=',').str.get_dummies(sep=',')
# Save the list of categories for future use
categories = categories_df.columns.values
# Merge it with our original dataframe
business_df = pd.merge(business_df, categories_df, left_index = True, right_index = True)

In [18]:
# join checkin and business collections 
# retrieve checkin documents only related to "Pittsburgh"
business_restaurant= business_df[business_df['Restaurants'] == 1]
business_id_name = business_restaurant[['business_id','name','categories']]
checkin_pit = pd.merge(checkin_df, business_id_name, how = 'inner', on = 'business_id')

checkin_pit.head()


,business_id,time,name,categories
0,JzB7NITHQ7gVHGVZ1ntgIQ,"{'Thursday': {'21:00': 1, '2:00': 2, '1:00': 5...",Primanti Bros.,"[Nightlife, Bars, American (New), Restaurants,..."
1,S7ECG2HNltSWFo1FcFTwFQ,"{'Thursday': {'22:00': 1, '23:00': 5, '0:00': ...",Outback Steakhouse,"[Steakhouses, Restaurants]"
2,oxb73l504iA68bWdHqymHQ,"{'Friday': {'20:00': 1, '16:00': 6, '23:00': 2...",Denny's,"[Breakfast & Brunch, Diners, Restaurants, Amer..."
3,minFxQJgD6lyHndwU-_IUw,"{'Friday': {'22:00': 1, '15:00': 1}, 'Thursday...",Boston Market,"[Comfort Food, Restaurants, Caterers, American..."
4,iuIkzvQRb4DVWOMhE4GI3g,"{'Monday': {'19:00': 1, '16:00': 4, '17:00': 3...",Ya Fei,"[Asian Fusion, Sushi Bars, Restaurants, Chinese]"


In [20]:
checkin_pit.to_json("checkin_restaurant.json",orient='records',lines = True)

In [21]:
business_id_name = business_df[['business_id','name','categories']]
checkin_pit = pd.merge(checkin_df, business_id_name, how = 'inner', on = 'business_id')

checkin_pit.head()

,business_id,time,name,categories
0,JzB7NITHQ7gVHGVZ1ntgIQ,"{'Thursday': {'21:00': 1, '2:00': 2, '1:00': 5...",Primanti Bros.,"[Nightlife, Bars, American (New), Restaurants,..."
1,exSX-Sa63Lh8lLtPCs_tqw,"{'Friday': {'5:00': 1}, 'Monday': {'17:00': 1}...",Spirit Airlines,"[Hotels & Travel, Transportation, Airlines]"
2,lH4NsIYCiHoLxEZvb4I2fg,"{'Monday': {'23:00': 1, '21:00': 1}, 'Saturday...",Disney Store,"[Children's Clothing, Toy Stores, Shopping, Fa..."
3,lBj-NFY7piJjD-zmOoBeyw,"{'Sunday': {'15:00': 1}, 'Friday': {'14:00': 1...",Men's Wearhouse,"[Shopping, Fashion, Formal Wear, Men's Clothin..."
4,S7ECG2HNltSWFo1FcFTwFQ,"{'Thursday': {'22:00': 1, '23:00': 5, '0:00': ...",Outback Steakhouse,"[Steakhouses, Restaurants]"


In [22]:
checkin_pit.to_json("checkin_pit.json",orient='records',lines = True)

In [5]:
# Format the votes as a list of dict objects
time_dict = [{'time': x} for x in checkin_pit['time'].values]


# Create a DataFrame with json_normalize
time_df = pd.io.json.json_normalize(time_dict)

# Merge the formatted votes_df with our original review_df
checkin_pit = pd.merge(checkin_pit, time_df, left_index = True, right_index = True)

# Drop the votes column
checkin_pit = checkin_pit.drop('time', axis=1)

In [6]:
checkin_pit.head(2)

,business_id,name,time.Friday.0:00,time.Friday.10:00,time.Friday.11:00,time.Friday.12:00,time.Friday.13:00,time.Friday.14:00,time.Friday.15:00,time.Friday.16:00,...,time.Wednesday.22:00,time.Wednesday.23:00,time.Wednesday.2:00,time.Wednesday.3:00,time.Wednesday.4:00,time.Wednesday.5:00,time.Wednesday.6:00,time.Wednesday.7:00,time.Wednesday.8:00,time.Wednesday.9:00
0,JzB7NITHQ7gVHGVZ1ntgIQ,Primanti Bros.,7.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,8.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,exSX-Sa63Lh8lLtPCs_tqw,Spirit Airlines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
